In [1]:
import numpy as np
import os
from tqdm import tqdm

# === Paths ===
feature_save_dir = "saved_feature_vectors/"
exemplar_save_dir = "exemplar_features/"
os.makedirs(exemplar_save_dir, exist_ok=True)

# === Load features for all 3 motion models ===
feature_files = {
    "y_speed": "y_speed_feature_updated.npy",
    "y_ang": "y_ang_features_updated.npy",
    "y_bkg": "y_bkg_features_updated.npy"
}

In [2]:
features = {}
for name, fname in feature_files.items():
    path = os.path.join(feature_save_dir, fname)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")
    data = np.load(path)
    features[name] = data.astype(np.float32)

# === Stack motion features into one combined vector ===
y_speed = features["y_speed"]
y_ang   = features["y_ang"]
y_bkg   = features["y_bkg"]

assert y_speed.shape[0] == y_ang.shape[0] == y_bkg.shape[0], "Mismatch in sample counts"

# Concatenate into combined motion vector (N, 768)
combined_features = np.concatenate([y_speed, y_ang, y_bkg], axis=1)

In [5]:
# === Compute Normalization Constants ===
def compute_Z_max(X, sample_size=1000):
    idx = np.random.choice(len(X), min(sample_size, len(X)), replace=False)
    sampled = X[idx]
    max_dist = 0
    for i in range(len(sampled)):
        for j in range(i + 1, len(sampled)):
            dist = np.linalg.norm(sampled[i] - sampled[j])
            if dist > max_dist:
                max_dist = dist
    return max_dist

Z_speed = compute_Z_max(y_speed)
Z_ang   = compute_Z_max(y_ang)
Z_bkg   = compute_Z_max(y_bkg)

Z_speed = Z_speed if Z_speed > 0 else 1
Z_ang   = Z_ang if Z_ang > 0 else 1
Z_bkg   = Z_bkg if Z_bkg > 0 else 1

print(f"[Z] Normalization constants → Z_ang: {Z_ang:.4f}, Z_speed: {Z_speed:.4f}, Z_bkg: {Z_bkg:.4f}")

[Z] Normalization constants → Z_ang: 4.1519, Z_speed: 12.4417, Z_bkg: 16.7560


In [6]:
# === Distance function from paper ===
def motion_distance(v1, v2):
    d_ang   = np.linalg.norm(v1[256:512] - v2[256:512]) / Z_ang
    d_speed = np.linalg.norm(v1[:256] - v2[:256]) / Z_speed
    d_bkg   = np.linalg.norm(v1[512:] - v2[512:]) / Z_bkg
    return d_ang + d_speed + d_bkg

In [7]:
# === Greedy exemplar selection (from paper) ===
def select_exemplars(features, threshold=1.0):
    exemplars = [features[0]]
    for i in tqdm(range(1, len(features)), desc="Selecting Exemplars"):
        dists = [motion_distance(features[i], ex) for ex in exemplars]
        if min(dists) > threshold:
            exemplars.append(features[i])
    return np.stack(exemplars)

In [8]:
# === Run selection ===
print("[🚀] Running exemplar selection based on combined motion features...")
exemplars = select_exemplars(combined_features, threshold=1.0)

[🚀] Running exemplar selection based on combined motion features...


Selecting Exemplars: 100%|████████████████████████████████████████████████████| 188739/188739 [08:17<00:00, 379.50it/s]


In [9]:
# === Save ===
np.save(os.path.join(exemplar_save_dir, "motion_exemplars.npy"), exemplars)
print(f"[✅] Saved {len(exemplars)} motion-based exemplars to: motion_exemplars.npy")

[✅] Saved 567 motion-based exemplars to: motion_exemplars.npy
